# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from io import StringIO
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

def insert_from_dataframe(df, insert_query):
    for i, row in df.iterrows():
        cur.execute(insert_query, list(row))
    conn.commit()

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data/')
type(song_files)

list

In [5]:
song_files_iterator = iter(song_files)
file_path = next(song_files_iterator)
df = pd.read_json(file_path, lines=True)
for song_file in song_files_iterator:
    df = df.append(pd.read_json(song_file, lines=True), ignore_index=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARMAC4T1187FB3FA4C,40.82624,"Morris Plains, NJ",-74.47995,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004
1,ARB29H41187B98F0EF,41.88415,Chicago,-87.63241,Terry Callier,342.56934,1,SOGNCJP12A58A80271,Do You Finally Need A Friend,1972
2,ARGIWFO1187B9B55B7,NaN,,NaN,Five Bolt Main,225.09669,1,SOPSWQW12A6D4F8781,Made Like This (Live),0
3,ARH4Z031187B9A71F2,40.73197,"Newark, NJ",-74.17418,Faye Adams,156.39465,1,SOVYKGO12AB0187199,Crazy Mixed Up World,1961
4,AR5KOSW1187FB35FF4,49.80388,Dubai UAE,15.47491,Elena,269.58322,1,SOZCTXZ12AB0182364,Setanta matins,0


## #1: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [6]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
artist_data = artist_data.drop_duplicates()
artist_data = artist_data.replace(np.nan, None, regex=True)
artist_data.columns = ['artist_id', 'name', 'location', 'latitude', 'longitude']
artist_data.head()

,artist_id,name,location,latitude,longitude
0,ARMAC4T1187FB3FA4C,The Dillinger Escape Plan,"Morris Plains, NJ",40.82624,-74.47995
1,ARB29H41187B98F0EF,Terry Callier,Chicago,41.88415,-87.63241
2,ARGIWFO1187B9B55B7,Five Bolt Main,,41.88415,-87.63241
3,ARH4Z031187B9A71F2,Faye Adams,"Newark, NJ",40.73197,-74.17418
4,AR5KOSW1187FB35FF4,Elena,Dubai UAE,49.80388,15.47491


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [7]:
insert_from_dataframe(artist_data, artist_table_insert)


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[['song_id','title', 'artist_id', 'year', 'duration']]
song_data = song_data.drop_duplicates()
song_data = song_data.replace(np.nan, None, regex=True)
song_data.head()

,song_id,title,artist_id,year,duration
0,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,ARMAC4T1187FB3FA4C,2004,207.77751
1,SOGNCJP12A58A80271,Do You Finally Need A Friend,ARB29H41187B98F0EF,1972,342.56934
2,SOPSWQW12A6D4F8781,Made Like This (Live),ARGIWFO1187B9B55B7,0,225.09669
3,SOVYKGO12AB0187199,Crazy Mixed Up World,ARH4Z031187B9A71F2,1961,156.39465
4,SOZCTXZ12AB0182364,Setanta matins,AR5KOSW1187FB35FF4,0,269.58322


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
insert_from_dataframe(song_data, song_table_insert)

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [10]:
log_files = get_files('data/log_data')

In [11]:
filepath = log_files[0]

In [12]:
log_files_iterator = iter(log_files)
file_path = next(log_files_iterator)
df = pd.read_json(file_path, lines=True)
for log_file in log_files_iterator:
    df = df.append(pd.read_json(log_file, lines=True), ignore_index=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Celeste,F,0,Williams,NaN,free,"Klamath Falls, OR",GET,Home,1.541078e+12,438,None,200,1541990217796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
1,Pavement,Logged In,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Gary Allan,Logged In,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,None,Logged In,Jacqueline,F,0,Lynch,NaN,paid,"Atlanta-Sandy Springs-Roswell, GA",GET,Home,1.540224e+12,389,None,200,1541990714796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [13]:
df = df[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Pavement,Logged In,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Gary Allan,Logged In,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
5,Charttraxx Karaoke,Logged In,Celeste,F,3,Williams,225.17506,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Fireflies,200,1541990752796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
6,The Libertines,Logged In,Jacqueline,F,1,Lynch,179.53914,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.540224e+12,389,The Good Old Days,200,1541990842796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29


In [14]:
# Parsing the ts column as datetime into an Series object from panda, then creating the DataFrame
tf = pd.DataFrame({
    'start_time': pd.to_datetime(df['ts'], unit='ms')
})

# Creating new columns

tf['hour'] = tf['start_time'].dt.hour
tf['day'] = tf['start_time'].dt.day
tf['week'] = tf['start_time'].dt.week
tf['month'] = tf['start_time'].dt.month
tf['year'] = tf['start_time'].dt.year
tf['weekday'] = tf['start_time'].dt.weekday

tf = tf.drop_duplicates()

tf.head()

,start_time,hour,day,week,month,year,weekday
1,2018-11-12 02:37:38.796,2,12,46,11,2018,0
2,2018-11-12 02:37:44.796,2,12,46,11,2018,0
3,2018-11-12 02:42:21.796,2,12,46,11,2018,0
5,2018-11-12 02:45:52.796,2,12,46,11,2018,0
6,2018-11-12 02:47:22.796,2,12,46,11,2018,0


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [15]:
insert_from_dataframe(tf, time_table_insert)

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [16]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df = user_df.drop_duplicates()
user_df = user_df[user_df['userId'] != '']
user_df.columns = ['user_id', 'first_name', 'last_name', 'gender', 'level']
user_df.head()
# duplicated_ids = [49,80,88,16,29,36,15,85]
# new_df = user_df[user_df[['user_id']].duplicated()]
# new_df
# new2_df = user_df[user_df['user_id'].isin(duplicated_ids)]
# new2_df

,user_id,first_name,last_name,gender,level
1,10,Sylvie,Cruz,F,free
2,53,Celeste,Williams,F,free
6,29,Jacqueline,Lynch,F,paid
12,68,Jordan,Rodriguez,F,free
18,94,Noah,Chavez,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [17]:
insert_from_dataframe(user_df, user_table_insert)

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [18]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (index, pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
    
    
#         songplay_id,
#         start_time,
#         user_id,
#         level,
#         song_id,
#         artist_id,
#         session_id,
#         location,
#         user_agent 
        

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [19]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.